<a href="https://colab.research.google.com/github/patty1997/Deep_Learning_Projects/blob/main/Resnet_SVM_Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import os
import seaborn as sns

from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
# from keras.preprocessing import image_dataset_from_directory
from keras.applications.resnet import ResNet50, preprocess_input
from keras.initializers import he_uniform, glorot_uniform
from keras.optimizers import SGD,Adam,RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.regularizers import l1,l2,l1_l2

In [ ]:
train_dir = "/content/drive/MyDrive/Colab Notebooks/Data/train_set"
validation_dir = "/content/drive/MyDrive/Colab Notebooks/Data/test_set"
test_dir = "/content/drive/MyDrive/Colab Notebooks/Data/random_test_set"

In [ ]:
random_tests =[]
for img_path in glob.glob(os.path.join(test_dir, "*.JPG")):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (224,224))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    random_tests.append(img)

In [ ]:
train_datagen = ImageDataGenerator(
                                    rotation_range = 15,
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    zoom_range = 0.2,
                                    preprocessing_function = preprocess_input,
                                    horizontal_flip = True
)

validation_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    class_mode = 'binary',
                                                    batch_size = 1396,
                                                    target_size = (224,224))

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                        class_mode = 'binary',
                                                        batch_size = 598,
                                                        target_size = (224,224))



Found 1396 images belonging to 2 classes.
Found 598 images belonging to 2 classes.


In [ ]:
x_train, y_train = train_generator.next()
x_test, y_test = validation_generator.next()
print(x_train.shape)
print(y_train.shape)

(1396, 224, 224, 3)
(1396,)


In [ ]:
resnet50_model = ResNet50(input_shape=(224,224,3),
                                 include_top = False,
                                 weights = 'imagenet')
for layer in resnet50_model.layers:
  layer.trainable = False
resnet50_model.summary()

94781440/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
__________________________

In [ ]:
feature_extractor  = Sequential([resnet50_model,Flatten()])

In [ ]:
X_for_SVM = feature_extractor.predict(x_train)
x_test_feature = feature_extractor.predict(x_test)

In [ ]:
from sklearn import svm
model = svm.LinearSVC(max_iter = 500)
model.fit(X_for_SVM, y_train)
prediction = model.predict(x_test_feature)

In [ ]:
from sklearn import metrics
print("Accuracy = ", metrics.accuracy_score(y_test, prediction))
print("F1Score = ", metrics.f1_score(y_test, prediction))

Accuracy =  0.9966555183946488
F1Score =  0.9966555183946488


In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, prediction)
print(cm)

[[298   1]
 [  1 298]]
